# Cách sử dụng milvus

In [1]:
from langchain_community.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="keepitreal/vietnamese-sbert", model_kwargs={"trust_remote_code": True})

c:\Users\PC\Desktop\Chatbot\.venv\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\PC\Desktop\Chatbot\.venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\PC\Desktop\Chatbot\.venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader, TextLoader

class CustomTextLoader(TextLoader):
    def __init__(self, file_path: str, encoding: str = 'utf-8'):
        super().__init__(file_path)
        self.encoding = encoding

    def _load(self):
        with open(self.file_path, 'r', encoding=self.encoding) as f:
            text = f.read()
        return text
txt_data_path = "C:/Users/PC/Desktop/Chatbot/chatbot/data"  # Thay thế bằng đường dẫn tới thư mục chứa file txt
loader = DirectoryLoader(txt_data_path, glob="*.txt", loader_cls=lambda file_path: CustomTextLoader(file_path, encoding='utf-8'))
documents = loader.load()

# Chia nhỏ văn bản thành các chunk
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

### cách kết nối với milvuss trong docker 

In [3]:
from langchain_milvus import Milvus

# The easiest way is to use Milvus Lite where everything is stored in a local file.
# If you have a Milvus server you can use the server URI such as "http://localhost:19530".
URI = "http://localhost:19530"

vector_store = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": URI},
)

In [4]:
from langchain_core.documents import Document

vector_store_saved = Milvus.from_documents(
    chunks,
    embeddings,
    collection_name="langchain_example",
    connection_args={"uri": URI},
)

In [5]:
vector_store_loaded = Milvus(
    embeddings,
    connection_args={"uri": URI},
    collection_name="langchain_example",
)

### Kiểm tra dữ liệu đã lưu trữ trong Milvus

In [6]:
query_result = vector_store_loaded.similarity_search("Điều 1. Phạm vi điều chỉnh", k=2) 
# In kết quả truy vấn
for i, result in enumerate(query_result):
    print(f"Result {i+1}: {result.page_content[:200]}...")

Result 1: Luật số: 91/2015/QH13
Hà Nội,ngày 28 tháng 12 năm 2015
BỘ LUẬT DÂN SỰ
Điều 185. Bảo vệ việc chiếm hữu
Trường hợp việc chiếm hữu bị người khác xâm phạm thì người chiếm hữu có
quyền yêu cầu người có hàn...
Result 2: Luật số: 91/2015/QH13
Hà Nội,ngày 28 tháng 12 năm 2015
BỘ LUẬT DÂN SỰ
Điều 185. Bảo vệ việc chiếm hữu
Trường hợp việc chiếm hữu bị người khác xâm phạm thì người chiếm hữu có
quyền yêu cầu người có hàn...


### Lấy Embeddings từ Milvus

In [7]:
# Truy xuất embedding từ Milvus
query_embeddings = vector_store_loaded.similarity_search_with_score("Phạm vi điều chỉnh", k=3)  # Ví dụ truy vấn

# In các embeddings từ kết quả truy vấn
for i, (doc, score) in enumerate(query_embeddings):
    print(f"Result {i+1} - Embedding Score: {score}")
    print(f"Document: {doc.page_content[:200]}...")  # In nội dung văn bản


Result 1 - Embedding Score: 89.29641723632812
Document: Luật số: 91/2015/QH13
Hà Nội,ngày 28 tháng 12 năm 2015
BỘ LUẬT DÂN SỰ
Điều 185. Bảo vệ việc chiếm hữu
Trường hợp việc chiếm hữu bị người khác xâm phạm thì người chiếm hữu có
quyền yêu cầu người có hàn...
Result 2 - Embedding Score: 89.29641723632812
Document: Luật số: 91/2015/QH13
Hà Nội,ngày 28 tháng 12 năm 2015
BỘ LUẬT DÂN SỰ
Điều 185. Bảo vệ việc chiếm hữu
Trường hợp việc chiếm hữu bị người khác xâm phạm thì người chiếm hữu có
quyền yêu cầu người có hàn...
Result 3 - Embedding Score: 89.29641723632812
Document: Luật số: 91/2015/QH13
Hà Nội,ngày 28 tháng 12 năm 2015
BỘ LUẬT DÂN SỰ
Điều 185. Bảo vệ việc chiếm hữu
Trường hợp việc chiếm hữu bị người khác xâm phạm thì người chiếm hữu có
quyền yêu cầu người có hàn...


## Hiển thị dữ liệu đã lưu trữ và embeddings tương ứng 

In [8]:
from pymilvus import connections, Collection

# Kết nối tới Milvus
connections.connect("default", host="localhost", port="19530")

# Tải collection
collection = Collection(name="langchain_example")  # Tên collection mà bạn đã lưu
collection.load()


### Dữ liệu gồm các trường và kiểu dữ liệu như thế nào ?

In [9]:
# Xem schema của collection
print(collection.schema)

{'auto_id': True, 'description': '', 'fields': [{'name': 'source', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535}}, {'name': 'text', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535}}, {'name': 'pk', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 768}}], 'enable_dynamic_field': False}


In [10]:
# Truy vấn toàn bộ dữ liệu trong collection với giới hạn số lượng kết quả
results = collection.query(expr="pk >= 0", output_fields=["pk", "text", "vector", "source"], limit=100)

# Hiển thị kết quả
for result in results:
    print(result)


{'pk': 451929499552452121, 'text': 'Nguyễn Khắc Huy sinh ngày 30/07/2002, là một kỹ sư AI trẻ tuổi và tài năng. Với chuyên môn của mình, Huy đang làm việc trên một chatbot hỏi đáp dựa trên dữ liệu luật pháp tại Việt Nam, giúp người dùng tiếp cận và hiểu rõ hơn về các quy định và điều luật. Công việc này không chỉ đòi hỏi sự am hiểu sâu sắc về công nghệ mà còn cần một kiến thức vững chắc về luật pháp, thể hiện sự đa tài và nhiệt huyết của Huy trong việc ứng dụng công nghệ vào các lĩnh vực quan trọng của cuộc sống.\nNguyên Linh An sinh ngày 30/07/2000, là một giáo viên tiếng Anh nhiệt huyết và giàu kinh nghiệm, hiện đang giảng dạy ở cấp 3 tại Tuyên Quang. Với đam mê và sự cống hiến cho ngành giáo dục, Linh An không chỉ giúp học sinh nắm vững kiến thức ngôn ngữ mà còn truyền cảm hứng học tập và phát triển kỹ năng giao tiếp tiếng Anh trong cuộc sống hàng ngày. Sự tận tâm và sáng tạo trong giảng dạy của Linh An đã để lại dấu ấn tích cực trong lòng nhiều thế hệ học sinh và hiện tại cô đang l